_____
# MAIN SIM

In [1]:
from pprint import pprint
import pandas as pd
import random
import chromadb
from chromadb.utils import embedding_functions

# Constants
NEW_VIDEO_WEIGHT = 10
STARTING_WEIGHT = 10

LIKE = 1
SHARE = 2
WATCH = 1
LOOP = 1

USER_MIN_INTEREST = 4
USER_MAX_INTEREST = 8
COUNT_QUEUED_VIDEOS = 5

# AI-Related Constants
OPENAI_API_KEY = ""
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

In [2]:
# Initialize ChromaDB client and collection
chroma_client = chromadb.PersistentClient(path="db")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_EMBEDDING_MODEL
)

collection = chroma_client.get_or_create_collection(
    name="Edgur_Video_DB_Vectorstore",
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"},
)

# Load data from Google Sheets
DB_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSAE2tBAnAdXsxk9a9YClFN7MSEVhzEmJD01ewwtooMLxL-Ilod26EbdD8sZeZk0ybiqD-jqT-9RZbn/pub?gid=497214901&single=true&output=csv"
df = pd.read_csv(DB_URL)

In [3]:
# Functions

def create_user_interest_dict(user_interest_list):
    if len(user_interest_list) < USER_MIN_INTEREST:
        raise ValueError("You must have at least 4 interests")
    elif len(user_interest_list) > USER_MAX_INTEREST:
        raise ValueError("You may have 8 interests at most")

    user_dict_interest = {interest: STARTING_WEIGHT for interest in user_interest_list}
    user_dict_interest["Random"] = STARTING_WEIGHT
    return user_dict_interest

def observe_action_taken(interest_tag, liked=False, shared=False, watched=False, loop_count=0):
    Actions = {"Like": LIKE, "Share": SHARE, "Watch": WATCH, "Loop": LOOP}
    total_action_points = 0
    if liked:
        total_action_points += Actions["Like"]
    if shared:
        total_action_points += Actions["Share"]
    if watched:
        total_action_points += Actions["Watch"]
    total_action_points += loop_count * Actions["Loop"]
    return total_action_points

def recalculate_percentages(user_dict_interest):
    total_weight = sum(user_dict_interest.values())
    user_dict_percentage = {interest_tag: (weight / total_weight) * 100 for interest_tag, weight in user_dict_interest.items()}
    return user_dict_percentage

def handle_interruptions(user_dict_percentage):
    interest_tag, interest_percentage = max(user_dict_percentage.items(), key=lambda item: item[1])
    if interest_percentage > 75:
        print(f"Have you been enjoying {interest_tag} so far?")
        response = input()
        if response.lower() == "yes":
            print("Recommend less of the same interest.")
    elif interest_percentage > 50:
        print("Suggesting relevant interests...")
    print(f"The highest tag is {interest_tag}. It dominates the interest weights by {interest_percentage}%.")

def note_actions_and_update_weights(user_dict_interest, interest_tag, liked=False, shared=False, watched=False, loop_count=0):
    total_action_points = observe_action_taken(interest_tag, liked, shared, watched, loop_count)
    if interest_tag in user_dict_interest:
        user_dict_interest[interest_tag] += total_action_points
    else:
        user_dict_interest[interest_tag] = STARTING_WEIGHT
    user_dict_percentage = recalculate_percentages(user_dict_interest)
    handle_interruptions(user_dict_percentage)
    user_dict_interest = dict(sorted(user_dict_interest.items(), key=lambda item: item[1], reverse=True)[:10])
    return user_dict_interest

def get_interest_tag_for_recommendation(user_dict_interest):
    user_dict_percentage = recalculate_percentages(user_dict_interest)
    random_value = random.randint(0, 100)
    cumulative_percentage = 0
    for interest_tag, percentage in user_dict_percentage.items():
        previous_cumulative_percentage = cumulative_percentage
        cumulative_percentage += percentage
        if random_value <= cumulative_percentage:
            break
    if interest_tag == "Random":
        interest_tag = random.choice(list(user_dict_percentage.keys()))
        return interest_tag
    return interest_tag

def add_to_watched(video_id, title, watched_dict):
    watched_dict[video_id] = title
    print(f'"{title}" with ID {video_id} has been added to watched videos.')
    return watched_dict

def show_to_user_again(video_id, title, watched_dict):
    watched_dict.pop(video_id)
    print(f'"{title}" with ID {video_id} will be shown to user again in the future.')
    return watched_dict

def add_to_queue(interest_tag, n_results=5):
    results = collection.query(query_texts=[interest_tag], n_results=n_results)
    print(f'Queried top {n_results} videos for tag "{interest_tag}"')
    for i in range(n_results):
        result_id = int(results["ids"][0][i])
        title = df.loc[df["video_id"] == result_id, "video_title"].values[0]
        tag = df.loc[df["video_id"] == result_id, "tags"].values[0].split(',')[0]
        if result_id not in watched:
            queue[result_id] = tag
            print(f'Added "{title}" with ID {result_id} and tag "{tag}" to the queue')
    return queue

def view_video(queue, user_dict_interest, watched_dict):
    if not queue:
        print("Queue is empty! Adding more videos...")
        new_interest_tag = get_interest_tag_for_recommendation(user_dict_interest)
        add_to_queue(new_interest_tag, COUNT_QUEUED_VIDEOS)

    video_id, interest_tag = next(iter(queue.items()))
    del queue[video_id]
    title = df.loc[df["video_id"] == video_id, "video_title"].values[0]
    add_to_watched(video_id, title, watched_dict)
    note_actions_and_update_weights(user_dict_interest, interest_tag, liked=True, watched=True)
    if len(queue) <= 2:
        print("\nAdding more videos to the queue...\n")
        new_interest_tag = get_interest_tag_for_recommendation(user_dict_interest)
        add_to_queue(new_interest_tag, COUNT_QUEUED_VIDEOS)
    return queue

In [4]:
# Main Simulation

def main_simulation(user_interests, initial_interest_tag, num_iterations=5):

    # Create a dictionary to store user interests
    user_dict_interest = create_user_interest_dict(user_interests)
    print(f"Initial user interests: {user_dict_interest}")

    # Add the initial interest tag to the queue
    queue = add_to_queue(initial_interest_tag)
    print(f"Initial queue: {queue}")

    # Iterate for the given number of iterations
    for i in range(num_iterations):
        print(f"\n=== Iteration {i+1} ===")

        # View the next video in the queue, update the queue, user interests, and watched videos
        queue = view_video(queue, user_dict_interest, watched)

        # Print the updated queue, user interests, and watched videos
        print(f"\nUpdated Queue: {queue}")
        print(f"Updated user interests: {user_dict_interest}")
        print(f"Watched Videos: {watched}")

        print("\n=========================\n")

In [5]:
# Example user interests and initial setup
user_interests = ["DevOps", "Tests (SAT)", "Machine Learning", "Cats", "Cooking"]
initial_interest_tag = "Shiba"

# Run the main simulation
main_simulation(user_interests, initial_interest_tag, 1)


Initial user interests: {'DevOps': 10, 'Tests (SAT)': 10, 'Machine Learning': 10, 'Cats': 10, 'Cooking': 10, 'Random': 10}
Queried top 5 videos for tag "Shiba"


NameError: name 'watched' is not defined

In [ ]:
# import yfinance as yf
# nvda = yf.Ticker("NVDA")
# nvda.info['regularMarketPrice']

price_now = 1200 # Change me

bought_at = 1138 # needs +75 to breakeven
my_current_net = (price_now - bought_at)
allsold = sum([291.25])
num_stocks = 3.76

must_be_positive = my_current_net * num_stocks - allsold

must_be_positive # sell when positive

-58.130000000000024

In [15]:
# Define the current price, the price at which you bought the stocks, and the number of stocks
price_now = 1287 # Change me
bought_at = 1138
num_stocks = 3.76

# Calculate the net gain per stock
my_current_net = (price_now - bought_at)

# Calculate the total amount you've already sold
allsold = sum([291.25])

# Calculate the net gain from selling all your stocks
must_be_positive = my_current_net * num_stocks - allsold

# Calculate the price at which you should sell your stocks to start making a profit
price_for_positive = bought_at + 78

# Print the results
print(f"Net gain from selling all stocks: {must_be_positive}")
print(f"Price to sell at for profit: {price_for_positive}")

profit_per_stock = price_now - price_for_positive
pprint(f"Profit per stock: {profit_per_stock}. Current profits: {(profit_per_stock * num_stocks) - allsold}")



Net gain from selling all stocks: 268.99
Price to sell at for profit: 1216
'Profit per stock: 71. Current profits: -24.29000000000002'


In [13]:
from pprint import pprint

# Define the current price, the price at which you bought the stocks, and the number of stocks
price_now = 128.80 # Adjust for stock split
bought_at = 1138.5 / 10 # Adjust for stock split
num_stocks = 3.76 * 10 # Adjust for stock split


# Calculate the total amount you've already sold
# Adjust for stock split by dividing the sold price by 10 and multiplying the quantity by 10
sold_1 = 121.359 * 0.24   # 291.25 / (10 * 10)
sold_stocks = sum([sold_1])

# Calculate the price at which you should sell your stocks to start making a profit
# Adjust for stock split by dividing the price by 10
price_for_positive = (bought_at + (78 / 10))
pprint(f"Price to sell at for profit: {price_for_positive}")

profit_per_stock = price_now - price_for_positive
pprint(f"Profit per stock: {profit_per_stock}. Current profits: {(profit_per_stock * num_stocks) - sold_stocks}")



'Price to sell at for profit: 121.64999999999999'
'Profit per stock: 7.15000000000002. Current profits: 239.71384000000072'


In [25]:
# Define the original price, the price at which you bought the stocks, and the number of stocks
price_now = 1287  # Current price before the split
bought_at = 1138
num_stocks = 3.76

# Adjust for the 10-for-1 stock split
price_now_split_adjusted = price_now / 10
bought_at_split_adjusted = bought_at / 10
num_stocks_split_adjusted = num_stocks * 10

# Amount sold and price at which it was sold before split
amount_sold = 291.25 + 257.81
price_sold_at = 1213.59 + (1289.1*10)
price_sold_at_split_adjusted = price_sold_at / 10

# Calculate the number of shares sold
num_shares_sold = amount_sold / price_sold_at_split_adjusted

# Remaining shares after the sale
remaining_shares = num_stocks_split_adjusted - num_shares_sold

# Calculate the net gain per stock
my_current_net = price_now_split_adjusted - bought_at_split_adjusted

# Calculate the total gain/loss from the shares sold
gain_from_sold_shares = (price_sold_at_split_adjusted - bought_at_split_adjusted) * num_shares_sold

# Calculate the net gain from remaining shares
net_gain_remaining_shares = (price_now_split_adjusted - bought_at_split_adjusted) * remaining_shares

# Calculate the overall net gain/loss
total_net_gain = gain_from_sold_shares + net_gain_remaining_shares

# Calculate the break-even price for remaining shares to cover the sold amount
break_even_price = bought_at_split_adjusted + (amount_sold / remaining_shares)

# Print the results
print(f"Net gain from selling all stocks: {total_net_gain}")
print(f"Break-even price to sell remaining shares for no loss: {break_even_price}")

profit_per_stock = price_now_split_adjusted - break_even_price
print(f"Profit per stock: {profit_per_stock}. Current profits: {(profit_per_stock * remaining_shares) + gain_from_sold_shares}")
print(net_gain_remaining_shares)

Net gain from selling all stocks: 1059.1999814953851
Break-even price to sell remaining shares for no loss: 128.55542434006193
Profit per stock: 0.14457565993805588. Current profits: 510.1399814953854
554.4397647574295
